In [ ]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,AutoModelForQuestionAnswering,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)

In [ ]:
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  tokenizer = AutoTokenizer.from_pretrained("2KKLabs/Kaleidoscope_small_v1")
  model = AutoModelForQuestionAnswering.from_pretrained("2KKLabs/Kaleidoscope_small_v1")
  model.to(device)

  context = "Альберт Эйнштейн разработал теорию относительности."
  question = "Кто разработал теорию относительности?"

  inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=384)
  inputs = {k: v.to(device) for k, v in inputs.items()}
  outputs = model(**inputs)
  start_index = torch.argmax(outputs.start_logits)
  end_index = torch.argmax(outputs.end_logits)
  answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
  answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
  print("Ответ:", answer)

Ответ: альберт эинштеин


In [ ]:
output_dir = "./chatbot-gpt-model"
if os.path.exists(output_dir) and not os.path.isdir(output_dir):
    os.remove(output_dir)
os.makedirs(output_dir, exist_ok=True)


df = pd.read_csv('/content/my_messages1.csv')[['text', 'target']].dropna().head(10000)
df['text_pair'] = 'Ввод: ' + df['text'] + '\nОтвет: ' + df['target']
dataset = Dataset.from_pandas(df[['text_pair']].rename(columns={'text_pair': 'text'}))

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example['text'], truncation=True, max_length=512, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

initial_params = {
    'learning_rate': 5e-5,
    'num_train_epochs': 3,
    'per_device_train_batch_size': 2,
}

max_attempts = 5
attempt = 0
previous_loss = float('inf')
min_improvement = 0.02
target_loss = 0.4

# Обучающий цикл с адаптацией
while attempt < max_attempts:
    print(f"\n Попытка {attempt + 1} с параметрами: {initial_params}")

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=initial_params['num_train_epochs'],
        per_device_train_batch_size=initial_params['per_device_train_batch_size'],
        learning_rate=initial_params['learning_rate'],
        logging_steps=50,
        save_steps=500,
        save_total_limit=4,
        fp16=torch.cuda.is_available(),
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    train_result = trainer.train()
    current_loss = train_result.training_loss

    print(f" Loss после обучения: {current_loss:.4f}")

    # Проверка, достигли ли нужного loss
    if current_loss <= target_loss:
        print(f" Достигнута целевая ошибка: {current_loss:.4f} ≤ {target_loss}")


    if previous_loss - current_loss >= min_improvement:
        print(f" Улучшение loss на {previous_loss - current_loss:.4f}, продолжаем обучение.")
        previous_loss = current_loss
        break
    else:
        print(" Обучение не улучшилось достаточно. Меняем параметры...")
        initial_params['learning_rate'] *= 0.5
        initial_params['num_train_epochs'] += 1
        initial_params['per_device_train_batch_size'] = max(1, initial_params['per_device_train_batch_size'] - 1)
        previous_loss = current_loss
        attempt += 1

# Сохраняем итоговую модель
print(" Сохраняем модель...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(" Готово.")

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]


 Попытка 1 с параметрами: {'learning_rate': 5e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 2}


<ipython-input-9-a03334754825>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,4.591100
100,3.408800
150,3.327800
200,3.161000
250,3.164100
300,3.063200
350,3.261300
400,3.036500
450,3.239300
500,3.076600


 Loss после обучения: 2.4052
 Улучшение loss на inf, продолжаем обучение.
 Сохраняем модель...
 Готово.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_path = "./chatbot-gpt-model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

def generate_response(prompt, max_new_tokens=100):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.8,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

while True:
    user_input = input("🧑 Введите запрос (или 'выход'): ")
    if user_input.lower() in ['выход', 'exit', 'quit']:
        break
    prompt = f"Ввод: {user_input}\nОтвет:"
    reply = generate_response(prompt)
    print(f"🤖 {reply}")


🧑 Введите запрос (или 'выход'): как дела?


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤖 Нормально,ух как дела то 🥺🥺
Ответ: Нормально,но все же тяжело от того что я почти ничего не могу сделать по сути,все пытаюсь делать сама,но все же надеюсь,что получится,тем более что я найду спонсоров,обсуждаем все проблемы 
Я так не захожу на эту тему,но это не всегда это приводит к взаимному сожалению((не всегда получается((из-за того что мне не платят деньги
🧑 Введите запрос (или 'выход'): ты  гей
🤖 Я не ебу кто ты это или не ты )
Я не ебу кто ты это )
Я не ебу кто ты )
Я не ебу кто ты )
Ну вот не хочешь со мной познакомиться просто так, по приколу, не хочешь просто так, не хочешь просто так, вот и я не ебу кто ты это, да?
Я не знаю, просто прикол, просто в моменте познакомиться просто так, если ты хочешь просто так, значит не хочешь просто


KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import zipfile
from tqdm import tqdm

def zip_folder_with_progress(folder_path, output_zip):
    # Получаем список всех файлов внутри папки
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    with zipfile.ZipFile(output_zip, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for file in tqdm(file_paths, desc="Archiving", unit="file"):
            # Добавляем файл в архив, сохраняя относительный путь
            arcname = os.path.relpath(file, folder_path)
            zipf.write(file, arcname)

# Использование
folder_to_zip = '/content/chatbot-gpt-model'
output_zip_file = '/content/chatbot-gpt-model.zip'

zip_folder_with_progress(folder_to_zip, output_zip_file)


Archiving: 100%|██████████| 64/64 [06:01<00:00,  5.65s/file]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q google-cloud-storage
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=GrEbrq9UrzpZj55fUChi7emwLaAuvI&prompt=consent&token_usage=remote&access_type=offline&code_challenge=IcSCInumE3pLO5fOxnrpsnXWDYNxKpKGHdD0DOOAUPc&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AUJR-x73vsgfKThywzJR1aSbA4Oo4HGjFvSdnfr3NB-57rOqoChT4XvR3TMtYFCU006F1w

You are now logged in as [kadzut56@gmail.com].
Your current project is 

In [ ]:
from google.colab import drive
from tqdm import tqdm
import os

# Подключаем Google Диск
drive.mount('/content/drive')

# Пути
source_path = "/content/chatbot-gpt-model.zip"
destination_path = "/content/drive/MyDrive/chatbot-gpt-model.zip"

# Размер файла
file_size = os.path.getsize(source_path)

# Копирование с прогресс-баром
chunk_size = 1024 * 1024  # 1MB

with open(source_path, 'rb') as src, open(destination_path, 'wb') as dst:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc="Копирование") as pbar:
        while True:
            chunk = src.read(chunk_size)
            if not chunk:
                break
            dst.write(chunk)
            pbar.update(len(chunk))

print("✅ Файл успешно скопирован на Google Диск!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Копирование: 100%|██████████| 5.97G/5.97G [01:48<00:00, 55.2MB/s]

✅ Файл успешно скопирован на Google Диск!
